# decappingQuantification.ipynb
## Marcus Viscardi,    April 15, 2024

Check out `README.md` for a more overarching view of what is happening here.

Specifically, the goal is to produce comparisons of true full-length reads (spanning out to the TSS) and the reads that have cleavages near their stop codons (and the other reads).

The largest choice to make here, is if we want to manually annotate a subset of genes to look at for this, or if we should rather look into a reference-based approach. I think the reference-based approach is likely to be more robust, but it will require a big up-front investment in time to get the reference data in a usable format.

Initially, I think I will start with the manual annotation approach, as it will be quicker to get started with and will allow me to get a better feel for the data and the types of things that I will be looking for.

In [ ]:
from typing import Tuple

import nanoporePipelineCommon as npCommon

import numpy as np
import pandas as pd

import re

import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from tqdm.auto import tqdm

from icecream import ic
from datetime import datetime

from pathlib import Path

from scipy.stats import mannwhitneyu, ks_2samp

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

def __time_formatter__():
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"ic: {now} | > "
ic.configureOutput(prefix=__time_formatter__)

_ = ic("Imports done!")

In [ ]:
gtf_parquet_path = Path("/data16/marcus/genomes/plus_cerENO2_elegansRelease100/230327_allChrs_plus-cerENO2.gtf.parquet")

gtf_df = pd.read_parquet(gtf_parquet_path)

current_dir = Path.cwd()

In [ ]:
obj_dict = {}
libs_to_run = [
    "oldN2",
    # "oldS6",
    "newerN2",
    "newerS6",
    "newerS5",
    "thirdN2",
    "thirdS5",
    "thirdS6",
    "temp25cN2",
    "temp25cS5",
    "temp25cS6",
    "temp25cS7",
]
for lib in libs_to_run:
    print(f"\nLoading {lib}...", end="")
    obj = npCommon.NanoporeRun(run_nickname=lib)
    obj_dict[lib] = obj
    print(" Done!")
    obj.load_mergedOnReads()
    obj.load_nmd_targets(return_sambamobj=False)

In [ ]:
def histo_plot_ends(test_gene, gtf_df, obj_dict, flanking_dist=200):
    gtf_for_gene = gtf_df.query("gene_name == @test_gene")
    start, end = gtf_for_gene.query("feature == 'gene'").loc[:, ['start', 'end']].values[0]
    
    strand = gtf_for_gene.query("feature == 'gene'").loc[:, 'strand'].values[0]
    print(f"Gene target ({test_gene}) on strand: {strand}")
    
    stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'start'].values
    start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'start'].values
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
    ax1, ax2 = axes
    
    sea.histplot(data=obj_dict['thirdN2'].mergedOnReads_df.query('gene_name == @test_gene and t5 == 0'),
                 x='chr_pos',
                 binwidth=10,
                 # stat='percent',
                 common_norm=False,
                 ax=ax1,
                 color='k',
                 alpha=0.5)
    ax1.set_ylabel('Unadapted Reads')
    sea.histplot(data=obj_dict['thirdN2'].mergedOnReads_df.query('gene_name == @test_gene and t5 == 1'),
                 x='chr_pos',
                 binwidth=10,
                 # stat='percent',
                 common_norm=False,
                 ax=ax2,
                 color='r',
                 alpha=0.5)
    ax2.set_ylabel('Adapted Reads')
    
    for stop_loc in stop_locations:
        ax1.axvline(stop_loc, color='r', linestyle='--')
        ax2.axvline(stop_loc, color='r', linestyle='--')
    for start_site in start_sites:
        ax1.axvline(start_site, color='g', linestyle='--')
        ax2.axvline(start_site, color='g', linestyle='--')
    
    ax1.set_xlim(start-flanking_dist, end+flanking_dist)
    ax2.set_xlim(start-flanking_dist, end+flanking_dist)
    
    plt.title(test_gene)
    plt.tight_layout()
    plt.show()

genes = [
    'rps-22',
    'ubl-1',
    'rpl-30',
    # 'eef-1A.1',  # rev  # Something weird is going on...
    'rla-1',  # rev
    'rps-25',  # rev
    'Y37E3.8',  # rev
    'rpl-10',
    'rla-0',
    'rpl-18',
    'rps-3',
    'rpl-38',
    'rpl-24.1',
    # NMD Targets:
    # 'rpl-30',
    # 'ubl-1',
    # 'rps-22',
    'hel-1',
    'rpl-7A',
    'ZK228.4',
    'R06C1.4',
    'rpl-12',
    'pqn-70',
    'rsp-5',
    'rpl-1',
    'C53H9.2',
    'rpl-3',
    'rsp-6',
    'rpl-26',
    'K08D12.3',
    'aly-3',
]
for gene_to_plot in genes:
    # histo_plot_ends(gene_to_plot, gtf_df, obj_dict, flanking_dist=100)
    print(gene_to_plot)

In [ ]:
UNANNOTATED_TSS_DICT = {  # these are manual annotations of stops that are not in the GTF file
    'rpl-7A': [4_390_579, 4_390_464],
}

UNANNOTATED_STOPS_DICT = {  # these are manual annotations of stops that are not in the GTF file
    'rps-22': [1_950_853],
    'rpl-30': [10_436_332],
}

def get_other_read_end(row):
    cigar = row['cigar']
    read_pos = row['chr_pos']
    if cigar is None:
        raise ValueError("Cigar is None!")
    if read_pos is None:
        raise ValueError("read_pos is None!")
    pattern = r"(\d*)([MDISN])"
    matches = re.findall(pattern, cigar)
    for match in matches:
        count, key = match
        if key in ['M', 'D', 'N']:
            read_pos += int(count)
    return read_pos

def near_a_tss_or_stop(row, _strand, tss_positions, stop_positions, col_for_pos='chr_pos', full_window_size=50, window_bias_downstream=0.50, make_window_correction=True) -> Tuple[bool, bool]:
    chr_pos = row[col_for_pos]
    if _strand == '-' and make_window_correction:
        window_bias_downstream = 1 - window_bias_downstream
    elif _strand != '+' and make_window_correction:
        raise ValueError(f"Strand must be '+' or '-' not {_strand}")
    
    near_tss = False
    near_stop = False
    chr_pos_upstream = chr_pos - full_window_size * (1 - window_bias_downstream)
    chr_pos_downstream = chr_pos + full_window_size * window_bias_downstream
    for tss in set(tss_positions):
        if chr_pos_upstream <= tss <= chr_pos_downstream:
            near_tss = True
            break
    for stop in set(stop_positions):
        if chr_pos_upstream <= stop <= chr_pos_downstream:
            near_stop = True
            break
    return near_tss, near_stop

def convert_near_tss_and_stop_to_hue(row, tss_color='g', stop_color='r', neither_color='k', both_color='y'):
    near_tss, near_stop = row['near_tss'], row['near_stop']
    if near_tss and near_stop:
        return both_color
    elif near_tss:
        return tss_color
    elif near_stop:
        return stop_color
    else:
        return neither_color

def plot_gene_data(gtf_df, test_gene, plot_lib_objs_dict, window_size=60,
                   up_v_down_fraction=0.25, number_of_bins=40,
                   flanking_gene_fraction=0.05, current_dir=None, print_stats=False,
                   title_override=None, save_name_override=None):
    
    extra_stops_dict = UNANNOTATED_STOPS_DICT
    
    extra_tss_dict = UNANNOTATED_TSS_DICT
    
    gtf_for_gene = gtf_df.query("gene_name == @test_gene")
    gene_start, gene_end = gtf_for_gene.query("feature == 'gene'").loc[:, ['start', 'end']].values[0]
    gene_length = gene_end - gene_start
    bin_width = gene_length / number_of_bins
    flanking_dist = int(gene_length * flanking_gene_fraction)
    
    strand = gtf_for_gene.query("feature == 'gene'").loc[:, 'strand'].values[0]
    if print_stats:
        print(f"Gene target ({test_gene}) on strand: {strand}")
    if strand == '-':
        if print_stats:
            print("Because this gene is on the reverse strand, we will have to calculate out the other end of the reads with the CIGAR strings. This will take slightly longer.")
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'end'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'end'].values
        up_v_down_fraction = 1 - up_v_down_fraction
        flipped = True
    else:
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'start'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'start'].values
        flipped = False
        
    stop_locations = list(stop_locations)
    start_sites = list(start_sites)
    
    if test_gene in extra_stops_dict:
        for stop_location in extra_stops_dict[test_gene]:
            stop_locations.append(stop_location)
    if test_gene in extra_tss_dict:
        for tss_location in extra_tss_dict[test_gene]:
            start_sites.append(tss_location)
    
    fig = plt.figure(figsize=(11, 1.5 * len(plot_lib_objs_dict)))
    
    gs = fig.add_gridspec(len(plot_lib_objs_dict), 5,
                          width_ratios=[4, 4, 1, 1, 1.5],
                          ) 
    
    for i, (lib, lib_obj) in enumerate(plot_lib_objs_dict.items()):

        if print_stats:
            print(f"\t{lib}:")
        ax1, ax2, ax3, ax4, ax5 = [fig.add_subplot(gs[i, j]) for j in range(5)]
        
        if i == 0:
            ax1.set_title("Unadapted 5' Ends")
            ax2.set_title("Adapted 5' Ends")
            ax3.set_title("Near TSS")
            ax4.set_title("Near Stop")
            ax5.set_title("Lib Stats")
        
        gene_df = lib_obj.mergedOnReads_df.query('gene_name == @test_gene').copy()
        gene_df.replace({'t5': {0: '-', 1: '+', '+': '+', '-': '-'}}, inplace=True)
        if flipped:
            gene_df.loc[:, 'chr_pos'] = gene_df.apply(get_other_read_end, axis=1)
        gene_df.loc[:, 'near_tss'], gene_df.loc[:, 'near_stop'] = zip(*gene_df.apply(near_a_tss_or_stop, args=(strand, start_sites, stop_locations),
            full_window_size=window_size, window_bias_downstream=up_v_down_fraction, axis=1))
        gene_df.loc[:, 'hue'] = gene_df.apply(convert_near_tss_and_stop_to_hue, axis=1)
        
        unadapted_df = gene_df.query('t5 == 0 or t5 == "-"')
        unadapted_value_counts = unadapted_df['near_tss'].value_counts(normalize=True)
        if print_stats:
            print(f"\t\tUnadapted Read Distribution (near a TSS):")
            print(f"\t\t\t True: {unadapted_value_counts.get(True, 0.0):%}")
            print(f"\t\t\tFalse: {unadapted_value_counts.get(False, 0.0):%}")
        if not unadapted_df.empty:
            sea.histplot(data=unadapted_df,
                         x='chr_pos',
                         hue='hue',
                         palette={'r': 'r', 'g': 'g', 'k': 'k', 'y': 'y'},
                         binwidth=bin_width,
                         ax=ax1,
                         edgecolor='k', linewidth=1.0,
                         )
            ax1.get_legend().remove()
            data_ct = pd.crosstab(index=gene_df['t5'], columns=gene_df['near_tss'])
            data_ct = data_ct.div(data_ct.sum(axis=1), axis=0) * 100
            data_ct.sort_index(ascending=False, inplace=True)
            
            color_dict = {True: 'g', False: 'k', None: 'y'}
            try:
                data_ct.plot(kind='bar',
                             stacked=True,
                             ax=ax3,
                             color=[color_dict[col] for col in data_ct.columns],
                             )
            except ValueError as e:
                print(f"Error: {e}")
                print(data_ct)
            ax3.set_ylabel ("") # ("% Near TSS")
            ax3.get_legend().remove()
        else:
            ax1.text(0.5, 0.5, "No Data", horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
            print(lib_obj.mergedOnReads_df.query('gene_name == @test_gene').head())
        ax1.set_ylabel(f"-- {lib} --")
        
        adapted_df = gene_df.query('t5 == 1 or t5 == "+"')
        adapted_value_counts = adapted_df['near_stop'].value_counts(normalize=True)
        if print_stats:
            print(f"\t\tAdapted Read Distribution (near a STOP):")
            print(f"\t\t\t True: {adapted_value_counts.get(True, 0.0):%}")
            print(f"\t\t\tFalse: {adapted_value_counts.get(False, 0.0):%}")
        if not adapted_df.empty:
            sea.histplot(data=adapted_df,
                         x='chr_pos',
                         hue='hue',
                         palette={'r': 'r', 'g': 'g', 'k': 'k', 'y': 'y'},
                         binwidth=bin_width,
                         ax=ax2,
                         edgecolor='k', linewidth=1.0,
                         )
            ax2.get_legend().remove()
            data_ct = pd.crosstab(index=gene_df['t5'], columns=gene_df['near_stop'])
            data_ct = data_ct.div(data_ct.sum(axis=1), axis=0) * 100
            data_ct.sort_index(ascending=False, inplace=True)
            
            color_dict = {True: 'r', False: 'k', None: 'y'}
            try:
                data_ct.plot(kind='bar',
                             stacked=True,
                             ax=ax4,
                             color=[color_dict[col] for col in data_ct.columns],
                             )
            except ValueError as e:
                print(f"Error: {e}")
                print(data_ct)
            ax4.set_ylabel("")  #("% Near Stop")
            ax4.get_legend().remove()
        else:
            ax2.text(0.5, 0.5, "No Data", horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
        ax2.set_ylabel(f"")
        
        ax1.set_xlim(gene_start-flanking_dist, gene_end+flanking_dist)
        ax2.set_xlim(gene_start-flanking_dist, gene_end+flanking_dist)
        ax1.set_xlabel(f"")
        ax2.set_xlabel(f"")
        ax1.set_xticklabels([])
        ax2.set_xticklabels([])
        
        ax3.set_xlabel(f"")
        ax4.set_xlabel(f"")
        ax3.set_xticklabels(['Unad', 'Ad'], rotation=0)
        ax4.set_xticklabels(['Unad', 'Ad'], rotation=0)
        
        precompute_transforms = {ax1: ax1.get_xaxis_transform(),
                                 ax2: ax2.get_xaxis_transform()}
        for stop_loc in set(stop_locations):
            for ax in [ax1, ax2]:
                ax.axvline(stop_loc, color='r', linestyle='--', alpha=0.5)
                ax.axvspan(stop_loc-window_size*up_v_down_fraction,
                           stop_loc+window_size*(1-up_v_down_fraction),
                           color='r', alpha=0.1)
                blend_transform = precompute_transforms[ax]
                ax.text(stop_loc, -0.1, "*", rotation=0,
                        verticalalignment='top', horizontalalignment='center',
                        transform=blend_transform,
                        fontsize=14)
        for start_site in set(start_sites):
            for ax in [ax1, ax2]:
                ax.axvline(start_site, color='g', linestyle='--', alpha=0.5)
                ax.axvspan(start_site-window_size*up_v_down_fraction,
                           start_site+window_size*(1-up_v_down_fraction),
                           color='g', alpha=0.1)
                blend_transform = precompute_transforms[ax]
                if flipped:
                    tss_icon = "<"
                else:
                    tss_icon = ">"
                ax.text(start_site, -0.05, tss_icon, rotation=0,
                        verticalalignment='top', horizontalalignment='center',
                        transform=blend_transform,
                        fontsize=12)
        
        
                # Create annotations to put in ax5
        ada_near_stop_count = adapted_df.query('near_stop').shape[0]
        ada_near_tss_count = adapted_df.query('near_tss').shape[0]
        unada_near_stop_count = unadapted_df.query('near_stop').shape[0]
        unada_near_tss_count = unadapted_df.query('near_tss').shape[0]
        ada_total_count = adapted_df.shape[0]
        unada_total_count = unadapted_df.shape[0]
        ada_stop_over_FL_string = f"{ada_near_stop_count}/{unada_near_tss_count} ({ada_near_stop_count/unada_near_tss_count:.2%})"
        ada_tss_over_FL_string = f"{ada_near_tss_count}/{unada_near_tss_count} ({ada_near_tss_count/unada_near_tss_count:.2%})"
        ax5.axis('off')
        plot_string = (
            f"Adapted Near Stop:   {ada_near_stop_count}\n"
            f"Adapted Near TSS:    {ada_near_tss_count}\n"
            f"Total Adapted:       {ada_total_count}\n"
            f"Unadapted Near Stop: {unada_near_stop_count}\n"
            f"Unadapted Near TSS:  {unada_near_tss_count}\n"
            f"Total Unadapted:     {unada_total_count}\n"
            f"Ada Stop/Unada FL: {ada_stop_over_FL_string}\n"
            f"Ada  TSS/Unada FL: {ada_tss_over_FL_string}"
        )
        ax5.text(-0.5, 0.4, plot_string,
                 fontsize=8,
                 fontfamily='monospace',
                 horizontalalignment='left',
                 verticalalignment='center',
                 # transform=ax5.transAxes,
                 )
    if isinstance(title_override, str):
        plt.suptitle(title_override)
    else:
        plt.suptitle(f"{test_gene} ({strand} strand)")
    plt.tight_layout()
    if current_dir is not None:
        if isinstance(save_name_override, str):
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{save_name_override}.png", dpi=300)
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{save_name_override}.svg")
        else:
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{test_gene}_5PrimeEnds.png", dpi=300)
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{test_gene}_5PrimeEnds.svg")
    plt.show()

# plot_libs = [
#     'oldN2',
#     'newerN2',
#     'newerS6',
#     'newerS5',
#     'thirdN2',
#     'thirdS5',
#     'thirdS6',
#     'temp25cN2',
#     'temp25cS5',
#     'temp25cS6',
#     'temp25cS7',
# ]

plot_libs = libs_to_run

plot_lib_objs_dict = {lib: obj_dict[lib] for lib in plot_libs}

plot_genes = genes[:]

for test_gene in plot_genes[:]:
    print(f"{test_gene:#^50}")
    try:
        # plot_gene_data(gtf_df, test_gene, plot_lib_objs_dict, current_dir=current_dir, print_stats=False)
        pass
    except Exception as e:
        print(f"Error: {e}")
        continue

In [ ]:
gene_ids_to_names_dict = gtf_df.query('feature == "gene"').set_index('gene_id')['gene_name'].to_dict()
gene_names_to_ids_dict = gtf_df.query('feature == "gene"').set_index('gene_name')['gene_id'].to_dict()
for gene_name in plot_genes:
    gene_id = gene_names_to_ids_dict[gene_name]
    print(f"{gene_name:>10} -> {gene_id}")
gene_ids = [gene_names_to_ids_dict[gene_name] for gene_name in plot_genes]

In [ ]:
import pysam

def load_simple_nmd_targets(nmd_targets_path, target_gene_ids_to_names_dict) -> pd.DataFrame:
    target_gene_names_to_ids_dict = {v: k for k, v in target_gene_ids_to_names_dict.items()}
    reads_dict = {}
    with pysam.AlignmentFile(nmd_targets_path, 'rb') as test_bam:
        for read in test_bam.fetch():
            read_assigned_gene_id = read.get_tag('gA')
            # print(read.query_name, read_assigned_gene_id, target_gene_ids_to_names_dict[read_assigned_gene_id], read.get_tag('t5'), read.get_tag('nC'))
            reads_dict[read.query_name] = {
                'gene_id': read_assigned_gene_id,
                'gene_name': target_gene_ids_to_names_dict[read_assigned_gene_id],
                't5': read.get_tag('t5'),
                'nmd_assignment': read.get_tag('nC'),
                'cigar': read.cigarstring,
                'chr_pos': read.reference_start,
                'chr': read.reference_name,
                'seq': read.query_sequence,
            }
    df = pd.DataFrame(reads_dict).T
    return df

nmd_genes = []

for lib, lib_obj in obj_dict.items():
    print(f"Loading {lib} NMD Targets from {lib_obj.nmd_targets_bam_path}...", end="")
    lib_obj.load_nmd_targets(return_sambamobj=False)
    print(" Done!")
    lib_obj.nmd_targets_df = load_simple_nmd_targets(lib_obj.nmd_targets_bam_path, gene_ids_to_names_dict)
    # print(lib_obj.nmd_targets_df.gene_name.unique())
    nmd_genes.extend(lib_obj.nmd_targets_df.gene_name.unique())
nmd_genes = list(set(nmd_genes))
print(nmd_genes)

In [ ]:
UNANNOTATED_TSS_DICT = {  # these are manual annotations of transcription start sites that are not in the GTF file
    'rpl-7A': [4_390_579, 4_390_464],
}

UNANNOTATED_STOPS_DICT = {  # these are manual annotations of stops that are not in the GTF file
    'rps-22': [1_950_853],  # cassette exon contains PTC
    'rpl-30': [10_436_332],  # extension of first exon contains PTC
    'rpl-7A': [4_389_880, 4_389_745],  # unannotated extension of 3rd exon contains PTC (second loc due to potential FS)
    'rsp-6': [7_790_576],  # longer unannotated form of cassette exon causes frame shift
    'C53H9.2': [1_833_467],  # shorter first exon leaves frame shift
    'pqn-70': [11_226_879],  # skipped exon leads to a frame change
    'rpl-1': [2_875_908],  # new orf started in the 3' extension of the first exon, causing a frame change
    'R06C1.4': [11_931_081],  # unannotated cassette exon contains a frame shift
    'rpl-3': [3_868_335],  # incorrectly annotated first intron 3' truncation (2nd exon extended) contains stop
    'hel-2': [8_327_970],  # later 5' SS adds to 3rd exon and contains PTC
    'rpl-12': [13_240_078],  # later 5' SS of second intron adds extension to 2nd exon with a PTC
    'aly-3': [12_123_859], # huge extension of second exon contains as PTC
}

def plot_gene_data3(gtf_df, test_gene, plot_lib_df_dict, window_size=100,
                   up_v_down_fraction=0.10, number_of_bins=40,
                   flanking_gene_fraction=0.05, current_dir=None, print_stats=False,
                   title_override=None, save_name_override=None, show_plot=True):
    
    min_reads_cutoff = 3
    
    extra_stops_dict = UNANNOTATED_STOPS_DICT
    extra_tss_dict = UNANNOTATED_TSS_DICT
    
    gtf_for_gene = gtf_df.query("gene_name == @test_gene")
    gene_start, gene_end = gtf_for_gene.query("feature == 'gene'").loc[:, ['start', 'end']].values[0]
    gene_length = gene_end - gene_start
    bin_width = gene_length / number_of_bins
    flanking_dist = int(gene_length * flanking_gene_fraction)
    
    strand = gtf_for_gene.query("feature == 'gene'").loc[:, 'strand'].values[0]
    if print_stats:
        print(f"Gene target ({test_gene}) on strand: {strand}")
    if strand == '-':
        if print_stats:
            print("Because this gene is on the reverse strand, we will have to calculate out the other end of the reads with the CIGAR strings. This will take slightly longer.")
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'end'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'end'].values
        up_v_down_fraction = 1 - up_v_down_fraction
        flipped = True
    elif strand == '+':
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'start'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'start'].values
        flipped = False
    else:
        raise ValueError(f"Strand must be '+' or '-' not '{strand}'")
        
    stop_locations = list(stop_locations)
    start_sites = list(start_sites)
    
    if test_gene in extra_stops_dict:
        for stop_location in extra_stops_dict[test_gene]:
            stop_locations.append(stop_location)
    if test_gene in extra_tss_dict:
        for tss_location in extra_tss_dict[test_gene]:
            start_sites.append(tss_location)
    
    fig = plt.figure(figsize=(11, 1.5 * len(plot_lib_df_dict)))
    
    gs = fig.add_gridspec(len(plot_lib_df_dict), 5,
                          width_ratios=[4, 4, 1, 1, 1.5],
                          )
    summary_dict = {}
    for i, (lib, lib_df) in enumerate(plot_lib_df_dict.items()):

        if print_stats:
            print(f"\t{lib}:")
        ax1, ax2, ax3, ax4, ax5 = [fig.add_subplot(gs[i, j]) for j in range(5)]
        
        if i == 0:
            ax1.set_title("Unadapted 5' Ends")
            ax2.set_title("Adapted 5' Ends")
            ax3.set_title("Near TSS")
            ax4.set_title("Near Stop")
            ax5.set_title("Lib Stats")
        
        gene_df = lib_df.query('gene_name == @test_gene').copy()
        gene_df.replace({'t5': {0: '-', 1: '+', '+': '+', '-': '-'}}, inplace=True)
        if flipped:
            gene_df.loc[:, 'chr_pos'] = gene_df.apply(get_other_read_end, axis=1)
        gene_df.loc[:, 'near_tss'], gene_df.loc[:, 'near_stop'] = zip(*gene_df.apply(near_a_tss_or_stop, args=(strand, start_sites, stop_locations),
            full_window_size=window_size,window_bias_downstream=up_v_down_fraction, make_window_correction=False, axis=1))
        gene_df.loc[:, 'hue'] = gene_df.apply(convert_near_tss_and_stop_to_hue, axis=1)
        
        unadapted_df = gene_df.query('t5 == "-"').copy()
        adapted_df = gene_df.query('t5 == "+"').copy()
        
        # Create annotations to put in ax5
        ada_near_stop_count = adapted_df.query('near_stop').shape[0]
        ada_near_tss_count = adapted_df.query('near_tss').shape[0]
        unada_near_stop_count = unadapted_df.query('near_stop').shape[0]
        unada_near_tss_count = unadapted_df.query('near_tss').shape[0]
        ada_total_count = adapted_df.shape[0]
        unada_total_count = unadapted_df.shape[0]
        try:
            ada_stop_over_FL_string = f"{ada_near_stop_count}/{unada_near_tss_count} ({ada_near_stop_count/unada_near_tss_count:.2%})"
            ada_tss_over_FL_string = f"{ada_near_tss_count}/{unada_near_tss_count} ({ada_near_tss_count/unada_near_tss_count:.2%})"
        except ZeroDivisionError:
            ada_stop_over_FL_string = f"{ada_near_stop_count}/{unada_near_tss_count} (X.XX%)"
            ada_tss_over_FL_string = f"{ada_near_tss_count}/{unada_near_tss_count} (X.XX%)"
        ax5.axis('off')
        plot_string = (
            f"Adapted Near Stop:   {ada_near_stop_count}\n"
            f"Adapted Near TSS:    {ada_near_tss_count}\n"
            f"Total Adapted:       {ada_total_count}\n"
            f"Unadapted Near Stop: {unada_near_stop_count}\n"
            f"Unadapted Near TSS:  {unada_near_tss_count}\n"
            f"Total Unadapted:     {unada_total_count}\n"
            f"Ada Stop/Unada FL: {ada_stop_over_FL_string}\n"
            f"Ada  TSS/Unada FL: {ada_tss_over_FL_string}"
        )
        summary_dict[lib] = {
            'Adapted Near Stop': ada_near_stop_count,
            'Adapted Near TSS': ada_near_tss_count,
            'Total Adapted': ada_total_count,
            'Unadapted Near Stop': unada_near_stop_count,
            'Unadapted Near TSS': unada_near_tss_count,
            'Total Unadapted': unada_total_count,
        }
        try:
            unadapted_value_counts = unadapted_df['near_tss'].value_counts(normalize=True)
            if print_stats:
                print(f"\t\tUnadapted Read Distribution (near a TSS):")
                print(f"\t\t\t True: {unadapted_value_counts.get(True, 0.0):%}")
                print(f"\t\t\tFalse: {unadapted_value_counts.get(False, 0.0):%}")
            if unadapted_df.shape[0] > min_reads_cutoff:
                sea.histplot(data=unadapted_df,
                             x='chr_pos',
                             hue='hue',
                             palette={'r': 'r', 'g': 'g', 'k': 'k', 'y': 'y'},
                             binwidth=bin_width,
                             ax=ax1,
                             edgecolor='k', linewidth=1.0,
                             )
                ax1.get_legend().remove()
            else:
                ax1.text(0.5, 0.5, "Insufficient Reads\nFor Plotting", horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
                print(lib_obj.mergedOnReads_df.query('gene_name == @test_gene').head())
            ax1.set_ylabel(f"-- {lib} --")
            
            data_ct = pd.crosstab(index=gene_df['t5'], columns=gene_df['near_tss'])
            data_ct = data_ct.div(data_ct.sum(axis=1), axis=0) * 100
            data_ct.sort_index(ascending=False, inplace=True)
            
            color_dict = {True: 'g', False: 'k', None: 'y'}
            try:
                data_ct.plot(kind='bar',
                             stacked=True,
                             ax=ax3,
                             color=[color_dict[col] for col in data_ct.columns],
                             )
            except ValueError as e:
                print(f"Error: {e}")
                print(data_ct)
            ax3.set_ylabel ("") # ("% Near TSS")
            ax3.get_legend().remove()
            
            adapted_value_counts = adapted_df['near_stop'].value_counts(normalize=True)
            if print_stats:
                print(f"\t\tAdapted Read Distribution (near a STOP):")
                print(f"\t\t\t True: {adapted_value_counts.get(True, 0.0):%}")
                print(f"\t\t\tFalse: {adapted_value_counts.get(False, 0.0):%}")
            if adapted_df.shape[0] > min_reads_cutoff:
                sea.histplot(data=adapted_df,
                             x='chr_pos',
                             hue='hue',
                             palette={'r': 'r', 'g': 'g', 'k': 'k', 'y': 'y'},
                             binwidth=bin_width,
                             ax=ax2,
                             edgecolor='k', linewidth=1.0,
                             )
                ax2.get_legend().remove()
            else:
                ax2.text(0.5, 0.5, "Insufficient Reads\nFor Plotting", horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
            
            ax2.set_ylabel(f"")
            
            data_ct = pd.crosstab(index=gene_df['t5'], columns=gene_df['near_stop'])
            data_ct = data_ct.div(data_ct.sum(axis=1), axis=0) * 100
            data_ct.sort_index(ascending=False, inplace=True)
            
            color_dict = {True: 'r', False: 'k', None: 'y'}
            try:
                data_ct.plot(kind='bar',
                             stacked=True,
                             ax=ax4,
                             color=[color_dict[col] for col in data_ct.columns],
                             )
            except ValueError as e:
                print(f"Error: {e}")
                print(data_ct)
            ax4.set_ylabel("")
            ax4.get_legend().remove()
            
            ax1.set_xlim(gene_start-flanking_dist, gene_end+flanking_dist)
            ax2.set_xlim(gene_start-flanking_dist, gene_end+flanking_dist)
            ax1.set_xlabel(f"")
            ax2.set_xlabel(f"")
            ax1.set_xticklabels([])
            ax2.set_xticklabels([])
            
            ax3.set_xlabel(f"")
            ax4.set_xlabel(f"")
            ax3.set_xticks([0 ,1], labels=['Unad', 'Ad'], rotation=0)
            ax4.set_xticks([0 ,1], labels=['Unad', 'Ad'], rotation=0)
            
            precompute_transforms = {ax1: ax1.get_xaxis_transform(),
                                     ax2: ax2.get_xaxis_transform()}
            for stop_loc in set(stop_locations):
                for ax in [ax1, ax2]:
                    ax.axvline(stop_loc, color='r', linestyle='--', alpha=0.5)
                    ax.axvspan(stop_loc-window_size*up_v_down_fraction,
                               stop_loc+window_size*(1-up_v_down_fraction),
                               color='r', alpha=0.1)
                    blend_transform = precompute_transforms[ax]
                    ax.text(stop_loc, -0.1, "*", rotation=0,
                            verticalalignment='top', horizontalalignment='center',
                            transform=blend_transform,
                            fontsize=14)
            for start_site in set(start_sites):
                for ax in [ax1, ax2]:
                    ax.axvline(start_site, color='g', linestyle='--', alpha=0.5)
                    ax.axvspan(start_site-window_size*up_v_down_fraction,
                               start_site+window_size*(1-up_v_down_fraction),
                               color='g', alpha=0.1)
                    blend_transform = precompute_transforms[ax]
                    if flipped:
                        tss_icon = "<"
                    else:
                        tss_icon = ">"
                    ax.text(start_site, -0.05, tss_icon, rotation=0,
                            verticalalignment='top', horizontalalignment='center',
                            transform=blend_transform,
                            fontsize=12)
            ax5.text(-0.5, 0.4, plot_string,
                     fontsize=8,
                     fontfamily='monospace',
                     horizontalalignment='left',
                     verticalalignment='center',
                     )
        except Exception as e:
            print(f"Error: {e}")
    if isinstance(title_override, str):
        plt.suptitle(title_override)
    else:
        plt.suptitle(f"{test_gene} ({strand} strand)")
    plt.tight_layout()
    if current_dir is not None:
        if isinstance(save_name_override, str):
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{save_name_override}.png", dpi=300)
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{save_name_override}.svg")
        else:
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{test_gene}_5PrimeEnds.png", dpi=300)
            plt.savefig(current_dir / 'plots' / '5PrimeEndsPerGene' / f"{test_gene}_5PrimeEnds.svg")
    if show_plot:
        plt.show()
    return summary_dict


def just_extract_gene_data(gtf_df, test_gene, plot_lib_df_dict, window_size=100,
                           up_v_down_fraction=0.25, number_of_bins=40,
                            flanking_gene_fraction=0.05, print_stats=False):    
    extra_stops_dict = UNANNOTATED_STOPS_DICT
    extra_tss_dict = UNANNOTATED_TSS_DICT
    
    gtf_for_gene = gtf_df.query("gene_name == @test_gene")
    gene_start, gene_end = gtf_for_gene.query("feature == 'gene'").loc[:, ['start', 'end']].values[0]
    gene_length = gene_end - gene_start
    
    strand = gtf_for_gene.query("feature == 'gene'").loc[:, 'strand'].values[0]
    if print_stats:
        print(f"Gene target ({test_gene}) on strand: {strand}")
    if strand == '-':
        if print_stats:
            print("Because this gene is on the reverse strand, we will have to calculate out the other end of the reads with the CIGAR strings. This will take slightly longer.")
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'end'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'end'].values
        up_v_down_fraction = 1 - up_v_down_fraction
        flipped = True
    else:
        stop_locations = gtf_for_gene.query("feature == 'stop_codon'").loc[:, 'start'].values
        start_sites = gtf_for_gene.query("feature == 'transcript'").loc[:, 'start'].values
        flipped = False
        
    stop_locations = list(stop_locations)
    start_sites = list(start_sites)
    
    if test_gene in extra_stops_dict:
        for stop_location in extra_stops_dict[test_gene]:
            stop_locations.append(stop_location)
    if test_gene in extra_tss_dict:
        for tss_location in extra_tss_dict[test_gene]:
            start_sites.append(tss_location)
    summary_dict_for_gene = {}
    for i, (lib, lib_df) in enumerate(plot_lib_df_dict.items()):

        if print_stats:
            print(f"\t{lib}:")
        
        gene_df = lib_df.query('gene_name == @test_gene').copy()
        gene_df.replace({'t5': {0: '-', 1: '+', '+': '+', '-': '-'}}, inplace=True)
        if flipped:
            gene_df.loc[:, 'chr_pos'] = gene_df.apply(get_other_read_end, axis=1)
        gene_df.loc[:, 'near_tss'], gene_df.loc[:, 'near_stop'] = zip(*gene_df.apply(near_a_tss_or_stop, args=(strand, start_sites, stop_locations),
            full_window_size=window_size,window_bias_downstream=up_v_down_fraction, axis=1))
        gene_df.loc[:, 'hue'] = gene_df.apply(convert_near_tss_and_stop_to_hue, axis=1)
        
        unadapted_df = gene_df.query('t5 == "-"').copy()
        adapted_df = gene_df.query('t5 == "+"').copy()
        
        # Create annotations to put in ax5
        ada_near_stop_count = adapted_df.query('near_stop').shape[0]
        ada_near_tss_count = adapted_df.query('near_tss').shape[0]
        unada_near_stop_count = unadapted_df.query('near_stop').shape[0]
        unada_near_tss_count = unadapted_df.query('near_tss').shape[0]
        ada_total_count = adapted_df.shape[0]
        unada_total_count = unadapted_df.shape[0]
        summary_dict_for_gene[lib] = {
            'Adapted Near Stop': ada_near_stop_count,
            'Adapted Near TSS': ada_near_tss_count,
            'Total Adapted': ada_total_count,
            'Unadapted Near Stop': unada_near_stop_count,
            'Unadapted Near TSS': unada_near_tss_count,
            'Total Unadapted': unada_total_count,
        }
    plt.show()
    return summary_dict_for_gene

# plot_libs = [
#     # 'oldN2',
#     'newerN2',
#     'newerS6',
#     'newerS5',
#     'thirdN2',
#     'thirdS5',
#     'thirdS6',
# ]

plot_libs = libs_to_run

plot_lib_objs_dict = {lib: obj_dict[lib] for lib in plot_libs}
overall_summaries_nested_dict = {}
for gene in nmd_genes[::-1]:
    # if gene != "rpl-7A":
    #     continue
    print(f"{gene:#^50}")
    gene_summary_dicts = {}
    target_nmd_types = ('nmd_target', 'ambiguous', 'non_nmd_target')
    subset_nmd_types_list = [target_nmd_types[0], target_nmd_types[:2], target_nmd_types[2], target_nmd_types[:]]
    subset_nmd_types_names = ['NMD_Targets', 'NMD_TargetsAndAmbiguous', 'NMD_NonTargets', 'All_NMD_Types']
    
    for subset_nmd_types, subset_nmd_types_name in zip(subset_nmd_types_list, subset_nmd_types_names):
        try:
            lib_df_dict_for_gene = {lib: lib_obj.nmd_targets_df.query('gene_name == @gene & nmd_assignment in @subset_nmd_types')
                                    for lib, lib_obj in plot_lib_objs_dict.items()}
            # summary_dict = plot_gene_data3(gtf_df, gene, lib_df_dict_for_gene, print_stats=False,
            #                                title_override=f"{gene}\nNMD Target Types: {subset_nmd_types}",
            #                                save_name_override=f"{gene}_{subset_nmd_types_name}",
            #                                current_dir=current_dir,)
            summary_dict = just_extract_gene_data(gtf_df, gene, lib_df_dict_for_gene, print_stats=False)
            gene_summary_dicts[subset_nmd_types_name] = summary_dict
        except Exception as e:
            print(f"Error: {e}")
            print(f"Gene: {gene}")
            print(f"NMD Types: {subset_nmd_types}")
            print(f"Continuing...")
    overall_summaries_nested_dict[gene] = gene_summary_dicts
overall_summaries_nested_dict

In [ ]:
from pprint import pprint
plotting_list_for_df = []

dict_index = 0
for gene, gene_dict in overall_summaries_nested_dict.items():
    for nmd_target_type, target_type_dict in gene_dict.items():
        print(f"{gene} ({nmd_target_type}):")
        for lib, lib_dict in target_type_dict.items():
            print(f"\t{lib}:")
            print(f"\t\tAdapted Near Stop:   {lib_dict['Adapted Near Stop']}")
            print(f"\t\tAdapted Near TSS:    {lib_dict['Adapted Near TSS']}")
            print(f"\t\tTotal Adapted:       {lib_dict['Total Adapted']}")
            print(f"\t\tUnadapted Near Stop: {lib_dict['Unadapted Near Stop']}")
            print(f"\t\tUnadapted Near TSS:  {lib_dict['Unadapted Near TSS']}")
            print(f"\t\tTotal Unadapted:     {lib_dict['Total Unadapted']}")
            try:
                print(f"\t\tAda Stop/Unada FL: {lib_dict['Adapted Near Stop']/lib_dict['Unadapted Near TSS']:.2%}")
            except ZeroDivisionError:
                print(f"\t\tAda Stop/Unada FL: -- NO UNADAPTED FL --")
            plotting_list_for_df.append(
                {
                    'gene_name': gene,
                    'nmd_target_type': nmd_target_type,
                    'lib': lib,
                    'adapted_near_stop': lib_dict['Adapted Near Stop'],
                    'adapted_near_tss': lib_dict['Adapted Near TSS'],
                    'total_adapted': lib_dict['Total Adapted'],
                    'unadapted_near_stop': lib_dict['Unadapted Near Stop'],
                    'unadapted_near_tss': lib_dict['Unadapted Near TSS'],
                    'total_unadapted': lib_dict['Total Unadapted'],
                }
            )

In [ ]:
plotting_summary_df = pd.DataFrame.from_records(plotting_list_for_df)
plotting_summary_df

In [ ]:
plotting_summary_df['nmd_active'] = plotting_summary_df.lib.str.contains('N2')
plotting_summary_df['ada_tss/unada_tss'] = plotting_summary_df['adapted_near_tss'] / plotting_summary_df['unadapted_near_tss']
plotting_summary_df['ada_stop/unada_tss'] = plotting_summary_df['adapted_near_stop'] / plotting_summary_df['unadapted_near_tss']
plotting_summary_df.nmd_target_type.unique()

In [ ]:
round_inf_to = 4
plotting_summary_df['ada_tss/unada_tss'] = plotting_summary_df['ada_tss/unada_tss'].replace({np.inf: round_inf_to, -np.inf: -round_inf_to})
plotting_summary_df['ada_stop/unada_tss'] = plotting_summary_df['ada_stop/unada_tss'].replace({np.inf: round_inf_to, -np.inf: -round_inf_to})

sea.histplot(data=plotting_summary_df.query("nmd_target_type == 'NMD_Targets'"),
             x='ada_stop/unada_tss',
             hue='lib',
             # element='step',
             bins=20,
             multiple='stack',
             fill=True,
             # discrete=True,
             log_scale=(True, False),
             )
plt.xscale('symlog')
plt.show()
sea.histplot(data=plotting_summary_df.query("nmd_target_type == 'NMD_Targets'"),
             x='ada_tss/unada_tss',
             hue='lib',
             # element='step',
             binwidth=0.0001,
             multiple='stack',
             fill=True,
             # discrete=True,
             # log_scale=(True, False),
             )
plt.xscale('symlog', linthresh=0.001)
plt.show()

In [ ]:
import seaborn.objects as so
import matplotlib.lines as mlines

linthresh = 0.0001
jitter_denominator = 4

# xcols = ['ada_tss/unada_tss', 'ada_stop/unada_tss']
xcols = ['ada_tss/total_tss', 'ada_stop/total_tss']
ycol = 'total_unadapted'

# lib_colors = {
#     'newerN2': 'red',
#     'newerS6': 'grey',
#     'newerS5': 'black',
#     'thirdN2': 'orange',
#     'thirdS6': 'grey',
#     'thirdS5': 'black',
# }
lib_colors = {
    'newerN2': 'grey',
    'newerS6': 'firebrick',
    'newerS5': 'darkorange',
    'thirdN2': 'black',
    'thirdS6': 'tomato',
    'thirdS5': 'orangered',
}

# fig, axes = plt.subplots(2, 2, figsize=(8, 8))
# ((ax1, ax2), (ax3, ax4)) = axes
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax1, ax2 = axes

nmd_types_to_plot = [
    'NMD_Targets',
    # 'NMD_NonTargets',
]
plot_df = plotting_summary_df.query("nmd_target_type in @nmd_types_to_plot").copy()

plot_df['total_tss'] = plot_df['adapted_near_tss'] + plot_df['unadapted_near_tss']
plot_df['total_stop'] = plot_df['adapted_near_stop'] + plot_df['unadapted_near_tss']

plot_df['ada_tss/total_tss'] = plot_df['adapted_near_tss'] / plot_df['total_tss']
plot_df['ada_stop/total_tss'] = plot_df['adapted_near_stop'] / plot_df['total_tss']

plot_df['lib_colors'] = plot_df.lib.map(lib_colors)
plot_df[xcols[0]] = plot_df[xcols[0]].transform(lambda x: np.random.uniform(
    -linthresh/jitter_denominator,
    linthresh/jitter_denominator,
) if x == 0 else x)
plot_df[xcols[1]] = plot_df[xcols[1]].transform(lambda x: np.random.uniform(
    -linthresh/jitter_denominator,
    linthresh/jitter_denominator,
) if x == 0 else x)

for gene in plot_df.gene_name.unique():
    sub_gene_plot_df = plot_df.query("gene_name == @gene")
    # sub_gene_plot_df.plot(x='ada_tss/unada_tss',
    #                       y='total_unadapted',
    #                       kind='line',
    #                       linestyle=':',
    #                       ax=ax1,
    #                       c='black',
    #                       alpha=0.25)
    # sub_gene_plot_df.plot(x='ada_stop/unada_tss',
    #                       y='total_unadapted',
    #                       kind='line',
    #                       linestyle=':',
    #                       ax=ax2,
    #                       c='black',
    #                       alpha=0.25)
    for lib in sub_gene_plot_df.lib.unique():
        sub_lib_plot_df = sub_gene_plot_df.query("lib == @lib")
        marker_types = ['o', '+']
        for i, nmd_target_type in enumerate(sub_lib_plot_df.nmd_target_type.unique()):
            sub_lib_type_plot_df = sub_gene_plot_df.query("lib == @lib & nmd_target_type == @nmd_target_type")
            sub_lib_type_plot_df.plot(x=xcols[0],
                                 y=ycol,
                                 kind='scatter',
                                 marker=marker_types[i],
                                 ax=ax1,
                                 c=sub_lib_type_plot_df['lib_colors'],
                                 alpha=0.5,
                                 zorder=100)
            sub_lib_type_plot_df.plot(x=xcols[1],
                                 y=ycol,
                                 kind='scatter',
                                 marker=marker_types[i],
                                 ax=ax2,
                                 c=sub_lib_type_plot_df['lib_colors'],
                                 alpha=0.5,
                                 zorder=100)
for ax in axes.flat:
    ax.set_xscale('symlog', linthresh=linthresh)
    ax.set_xlim(-linthresh, 10)
    ax.set_yscale('symlog', linthresh=10)
    legend = ax.get_legend()
    if legend is not None:
        legend.remove()
    ax.axvline(-linthresh/jitter_denominator,
               color='k', linestyle='--',
               alpha=0.25)
    ax.axvline(linthresh/jitter_denominator,
               color='k', linestyle='--',
               alpha=0.25)
    ax.annotate(' Points at zero jittered',
                xy=(-linthresh/jitter_denominator, 0),
                xytext=(-linthresh/jitter_denominator, 0),
                rotation=90,
                alpha=0.75,
                fontsize=8,
                horizontalalignment='right',
                verticalalignment='bottom',
                )

ax1.set_xlabel("decapped / total full length")
ax1.set_ylabel("Total Unadapted")
ax2.set_xlabel("cleaved / total full length")
ax2.set_ylabel("Total Unadapted")

ax1.set_ylim(0, 1e4)
ax2.set_ylim(0, 1e4)

legend_items = [mlines.Line2D([],
                              [],
                              color=lib_colors[lib],
                              marker='o',
                              linestyle='',
                              label=lib) for lib in plot_df.lib.unique()]
if len(plot_df.nmd_target_type.unique()) > 1:
    legend_items.extend([mlines.Line2D([],
                                       [],
                                       color='black',
                                       marker=marker,
                                       linestyle='',
                                       label=nmd_target_type) for nmd_target_type, marker in zip(plot_df.nmd_target_type.unique(), marker_types)])
    fig.legend(handles=legend_items, title='Library and NMD Type',
               loc='upper center',
               bbox_to_anchor=(0.0, 1.06, 1.0, 0.102),
               ncols=4,
               mode='expand',
               borderaxespad=0.0,
               facecolor='w',
               framealpha=1,
               )
else:
    fig.legend(handles=legend_items, title='Library',
               loc='upper center',
               bbox_to_anchor=(0.0, 1.00, 1.0, 0.102),
               ncols=6,
               mode='expand',
               borderaxespad=0.0,
               facecolor='w',
               framealpha=1,
               )

plt.tight_layout()
plt.draw()
save_dir = current_dir / 'plots' / 'decappingAndCleavage'
save_dir.mkdir(exist_ok=True, parents=True)
plt.savefig(save_dir / f"SummaryPlot{len(plot_df.nmd_target_type.unique())}.png",
            dpi=300, bbox_inches='tight')
plt.savefig(save_dir / f"SummaryPlot{len(plot_df.nmd_target_type.unique())}.svg",
            bbox_inches='tight')
plt.show()

In [ ]:
import seaborn.objects as so
import matplotlib.lines as mlines

linthresh = 1
jitter_denominator = 10

# xcols = ['ada_tss/unada_tss', 'ada_stop/unada_tss']
xcols = ['adapted_near_tss', 'adapted_near_stop']
ycol = 'total_tss'

lib_colors = {
    'newerN2': 'grey',
    'newerS6': 'firebrick',
    'newerS5': 'darkorange',
    'thirdN2': 'black',
    'thirdS6': 'tomato',
    'thirdS5': 'orangered',
}

# fig, axes = plt.subplots(2, 2, figsize=(8, 8))
# ((ax1, ax2), (ax3, ax4)) = axes
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax1, ax2 = axes

nmd_types_to_plot = [
    'NMD_Targets',
    'NMD_NonTargets',
]
plot_df = plotting_summary_df.query("nmd_target_type in @nmd_types_to_plot").copy()

plot_df['total_tss'] = plot_df['adapted_near_tss'] + plot_df['unadapted_near_tss']
plot_df['total_stop'] = plot_df['adapted_near_stop'] + plot_df['unadapted_near_tss']

plot_df['ada_tss/total_tss'] = plot_df['adapted_near_tss'] / plot_df['total_tss']
plot_df['ada_stop/total_tss'] = plot_df['adapted_near_stop'] / plot_df['total_tss']

plot_df['lib_colors'] = plot_df.lib.map(lib_colors)
plot_df[xcols[0]] = plot_df[xcols[0]].transform(lambda x: np.random.uniform(
    -linthresh/jitter_denominator,
    linthresh/jitter_denominator,
) if x == 0 else x)
plot_df[xcols[1]] = plot_df[xcols[1]].transform(lambda x: np.random.uniform(
    -linthresh/jitter_denominator,
    linthresh/jitter_denominator,
) if x == 0 else x)

for gene in plot_df.gene_name.unique():
    sub_gene_plot_df = plot_df.query("gene_name == @gene")
    for lib in sub_gene_plot_df.lib.unique():
        sub_lib_plot_df = sub_gene_plot_df.query("lib == @lib")
        marker_types = ['o', '+']
        for i, nmd_target_type in enumerate(sub_lib_plot_df.nmd_target_type.unique()):
            sub_lib_type_plot_df = sub_gene_plot_df.query("lib == @lib & nmd_target_type == @nmd_target_type")
            sub_lib_type_plot_df.plot(x=xcols[0],
                                 y=ycol,
                                 kind='scatter',
                                 marker=marker_types[i],
                                 ax=ax1,
                                 c=sub_lib_type_plot_df['lib_colors'],
                                 alpha=0.5,
                                 zorder=100)
            sub_lib_type_plot_df.plot(x=xcols[1],
                                 y=ycol,
                                 kind='scatter',
                                 marker=marker_types[i],
                                 ax=ax2,
                                 c=sub_lib_type_plot_df['lib_colors'],
                                 alpha=0.5,
                                 zorder=100)
for ax in axes.flat:
    ax.set_xscale('symlog', linthresh=linthresh)
    ax.set_xlim(-linthresh, 1e2)
    ax.set_yscale('symlog', linthresh=1)
    legend = ax.get_legend()
    if legend is not None:
        legend.remove()
    ax.axvline(-linthresh/jitter_denominator,
               color='k', linestyle='--',
               alpha=0.25)
    ax.axvline(linthresh/jitter_denominator,
               color='k', linestyle='--',
               alpha=0.25)
    ax.annotate(' Points at zero jittered',
                xy=(-linthresh/jitter_denominator, 0),
                xytext=(-linthresh/jitter_denominator, 0),
                rotation=90,
                alpha=0.75,
                fontsize=8,
                horizontalalignment='right',
                verticalalignment='bottom',
                )

ax1.set_xlabel("decapped")
ax1.set_ylabel("Total Near TSS")
ax2.set_xlabel("cleaved")
ax2.set_ylabel("Total Near TSS")

ax1.set_ylim(-1e0, 1e4)
ax2.set_ylim(-1e0, 1e4)

legend_items = [mlines.Line2D([],
                              [],
                              color=lib_colors[lib],
                              marker='o',
                              linestyle='',
                              label=lib) for lib in plot_df.lib.unique()]
if len(plot_df.nmd_target_type.unique()) > 1:
    legend_items.extend([mlines.Line2D([],
                                       [],
                                       color='black',
                                       marker=marker,
                                       linestyle='',
                                       label=nmd_target_type) for nmd_target_type, marker in zip(plot_df.nmd_target_type.unique(), marker_types)])
    fig.legend(handles=legend_items, title='Library and NMD Type',
               loc='upper center',
               bbox_to_anchor=(0.0, 1.06, 1.0, 0.102),
               ncols=4,
               mode='expand',
               borderaxespad=0.0,
               facecolor='w',
               framealpha=1,
               )
else:
    fig.legend(handles=legend_items, title='Library',
               loc='upper center',
               bbox_to_anchor=(0.0, 1.00, 1.0, 0.102),
               ncols=6,
               mode='expand',
               borderaxespad=0.0,
               facecolor='w',
               framealpha=1,
               )

plt.tight_layout()
plt.draw()
save_dir = current_dir / 'plots' / 'decappingAndCleavage'
save_dir.mkdir(exist_ok=True, parents=True)
plt.savefig(save_dir / f"SummaryPlot{len(plot_df.nmd_target_type.unique())}.png",
            dpi=300, bbox_inches='tight')
plt.savefig(save_dir / f"SummaryPlot{len(plot_df.nmd_target_type.unique())}.svg",
            bbox_inches='tight')
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

xcols = ['adapted_near_tss', 'adapted_near_stop']
ycol = 'total_tss'

lib_colors = {
    'newerN2': 'grey',
    'newerS6': 'firebrick',
    'newerS5': 'darkorange',
    'thirdN2': 'black',
    'thirdS6': 'tomato',
    'thirdS5': 'orangered',
}

nmd_types_to_plot = [
    'NMD_Targets',
    'NMD_NonTargets',
]
plot_df = plotting_summary_df.query("nmd_target_type in @nmd_types_to_plot").copy()

plot_df['total_tss'] = plot_df['adapted_near_tss'] + plot_df['unadapted_near_tss']
plot_df['total_stop'] = plot_df['adapted_near_stop'] + plot_df['unadapted_near_stop']

plot_df['ada_tss/total_tss'] = plot_df['adapted_near_tss'] / plot_df['total_tss']
plot_df['ada_stop/total_tss'] = plot_df['adapted_near_stop'] / plot_df['total_tss']

plot_df['lib_colors'] = plot_df.lib.map(lib_colors)

figures = []
# for xcol in xcols:
#     figures.append(px.scatter(plot_df,
#                               x=xcol,
#                               y=ycol,
#                               color='lib',
#                               symbol='nmd_target_type',
#                               color_discrete_map=lib_colors,
#                               symbol_sequence=['circle', 'cross'],
#                               log_x=True,
#                               log_y=True,
#                               title=f"{xcol} vs {ycol}",
#                               hover_name='gene_name',
#                               hover_data=[
#                                   'lib',
#                                   'nmd_target_type',
#                                   'total_unadapted',
#                                   'total_adapted',
#                                   'total_tss',
#                                   'total_stop',
#                                           ],
#                               
#                               )
#                      )
for i, xcol in enumerate(xcols):
    for gene_name in plot_df['gene_name'].unique():
        gene_df = plot_df[plot_df['gene_name'] == gene_name]
        figures.append(go.Scatter(
            x=gene_df[xcol],
            y=gene_df[ycol],
            mode='markers',
            marker=dict(
                color=gene_df['lib_colors'],
                symbol=gene_df['nmd_target_type'].map({'NMD_Targets': 'circle', 'NMD_NonTargets': 'cross'}),
                size=10
            ),
            name=gene_name,
            legendgroup=gene_name,
            showlegend=True if i == 0 else False,
        ))
fig = make_subplots(rows=1, cols=(len(xcols)))
for i, figure in enumerate(figures):
    fig.add_trace(figure, row=1, col=(i % len(xcols))+1)

# We need to add axis labels:
fig.update_xaxes(title_text="Decapped", row=1, col=1)
fig.update_xaxes(title_text="Cleaved", row=1, col=2)
fig.update_yaxes(title_text="Total Near TSS", row=1, col=1)
fig.update_yaxes(title_text="Total Near TSS", row=1, col=2)


fig.show(renderer='firefox')

In [ ]:
plotting_summary_df

# Work on 7/8/2024

Trying to get a more simple summary metric working. Going to be more aggressive with throwing out low depth libs.

The general idea is that the increase in reads near the TSS when we break NMD with smg-5 or smg-6 is due to the vast increase of the precursor NMD species. This would be expected if NMD targets were undergoing typical end of life cycle deadenylation and decapping.

If the decapping was somehow an artifact of the NMD knockouts, we'd expect the ratio between reads and TSS reads to differ between NMD targets and non-NMD targets.

An issue here, is that our ability to capture adapted near-TSS species is entirely reliant the knockdown of xrn-1 and the adapter ligation reaction efficacy.

One idea to avoid these sources of noise could be only looking within a library, comparing the ratio of decapped + adapted species on the NMD targets and the non-targets... ? Let's try:

In [ ]:
plot_libs = libs_to_run

window_size_for_search = 100

target_nmd_types = ('nmd_target', 'ambiguous', 'non_nmd_target')
subset_nmd_types_list = [target_nmd_types[0], target_nmd_types[1], target_nmd_types[2], target_nmd_types[:2], target_nmd_types[:]]
subset_nmd_types_names = ['NMD_Targets', 'Ambiguous_Reads', 'NMD_NonTargets', 'NMD_TargetsAndAmbiguous', 'All_Types']

plot_lib_objs_dict = {lib: obj_dict[lib] for lib in plot_libs}
overall_summaries_nested_dict = {}
for gene in nmd_genes[::-1]:
    # if gene != "rpl-7A":
    #     continue
    print(f"{gene:#^50}")
    gene_summary_dicts = {}
    
    for subset_nmd_types, subset_nmd_types_name in zip(subset_nmd_types_list, subset_nmd_types_names):
        try:
            lib_df_dict_for_gene = {lib: lib_obj.nmd_targets_df.query('gene_name == @gene & nmd_assignment in @subset_nmd_types')
                                    for lib, lib_obj in plot_lib_objs_dict.items()}
            # summary_dict = plot_gene_data3(gtf_df, gene, lib_df_dict_for_gene, print_stats=False,
            #                                title_override=f"{gene}\nNMD Target Types: {subset_nmd_types}",
            #                                save_name_override=f"{gene}_{subset_nmd_types_name}",
            #                                current_dir=current_dir,)
            summary_dict = just_extract_gene_data(gtf_df, gene, lib_df_dict_for_gene, print_stats=False, window_size=window_size_for_search)
            gene_summary_dicts[subset_nmd_types_name] = summary_dict
        except Exception as e:
            print(f"Error: {e}")
            print(f"Gene: {gene}")
            print(f"NMD Types: {subset_nmd_types}")
            print(f"Continuing...")
    overall_summaries_nested_dict[gene] = gene_summary_dicts

plotting_list_for_df = []

dict_index = 0
for gene, gene_dict in overall_summaries_nested_dict.items():
    for nmd_target_type, target_type_dict in gene_dict.items():
        print(f"{gene} ({nmd_target_type}):")
        for lib, lib_dict in target_type_dict.items():
            print(f"\t{lib}:")
            print(f"\t\tAdapted Near Stop:   {lib_dict['Adapted Near Stop']}")
            print(f"\t\tAdapted Near TSS:    {lib_dict['Adapted Near TSS']}")
            print(f"\t\tTotal Adapted:       {lib_dict['Total Adapted']}")
            print(f"\t\tUnadapted Near Stop: {lib_dict['Unadapted Near Stop']}")
            print(f"\t\tUnadapted Near TSS:  {lib_dict['Unadapted Near TSS']}")
            print(f"\t\tTotal Unadapted:     {lib_dict['Total Unadapted']}")
            try:
                print(f"\t\tAda Stop/Unada FL: {lib_dict['Adapted Near Stop']/lib_dict['Unadapted Near TSS']:.2%}")
            except ZeroDivisionError:
                print(f"\t\tAda Stop/Unada FL: -- NO UNADAPTED FL --")
            plotting_list_for_df.append(
                {
                    'gene_name': gene,
                    'nmd_target_type': nmd_target_type,
                    'lib': lib,
                    'adapted_near_stop': lib_dict['Adapted Near Stop'],
                    'adapted_near_tss': lib_dict['Adapted Near TSS'],
                    'total_adapted': lib_dict['Total Adapted'],
                    'unadapted_near_stop': lib_dict['Unadapted Near Stop'],
                    'unadapted_near_tss': lib_dict['Unadapted Near TSS'],
                    'total_unadapted': lib_dict['Total Unadapted'],
                }
            )

plotting_summary_df = pd.DataFrame.from_records(plotting_list_for_df)
plotting_summary_df

In [ ]:
summary_df = plotting_summary_df.copy()

lib_strain_dict = {'N2': 'wildtype',
                   'S5': 'smg-5',
                   'S6': 'smg-6',
                   'S7': 'smg-7'}

summary_df['strain'] = summary_df.lib.str[-2:].map(lib_strain_dict)

summary_df['total_near_tss'] = summary_df['adapted_near_tss'] + summary_df['unadapted_near_tss']
summary_df['total_near_stop'] = summary_df['adapted_near_stop'] + summary_df['unadapted_near_stop']
summary_df['total_reads'] = summary_df['total_adapted'] + summary_df['total_unadapted']

summary_df['frac_near_tss'] = summary_df['total_near_tss'] / summary_df['total_reads']
summary_df['frac_near_stop'] = summary_df['total_near_stop'] / summary_df['total_reads']

summary_df

In [ ]:
summary_gb  = summary_df.groupby(['gene_name', 'nmd_target_type', 'strain'])
df = summary_gb['frac_near_tss'].agg(['mean', 'std', 'sem'])

strain_dict = {}
for strain_val in summary_df['strain'].unique():
    strain_dict[strain_val] = df.loc[:, :, strain_val].rename(columns={'mean': f'{strain_val}_mean', 'std': f'{strain_val}_std', 'sem': f'{strain_val}_sem'})
remerge_df = pd.concat(strain_dict.values(), axis=1)

for strain_val in summary_df['strain'].unique():
    if strain_val == 'wildtype':
        continue
    remerge_df[f'{strain_val}/wildtype'] = remerge_df[f'{strain_val}_mean'] / remerge_df['wildtype_mean']
remerge_df[[col for col in remerge_df.columns if 'mean' in col]]  # This is very close, but now I am visually comparing each mean in NMD targets to the non-NMD targets. I want to do this via and actual comparison (maybe plot fractions or do a paired t-test between

In [ ]:
plot_df = remerge_df[[col for col in remerge_df.columns if 'mean' in col and 'smg-7' not in col]].loc[:, ['NMD_Targets', 'NMD_NonTargets'], :].reset_index()

fig = px.bar(plot_df,
             x='gene_name',
             y=[col for col in plot_df.columns if 'mean' in col and 'smg-7' not in col],
             color='nmd_target_type',
             barmode='group',
             )
fig.show(renderer='firefox')
plot_df

In [ ]:
wider_plot_dict = {}
for target_type in plot_df['nmd_target_type'].unique():
    target_type_df = plot_df.query("nmd_target_type == @target_type").drop(columns='nmd_target_type')
    wider_plot_dict[target_type] = target_type_df.rename(columns={col: f"{target_type}_{col}" for col in target_type_df.columns if 'mean' in col}).set_index('gene_name')
wider_plot_df = pd.concat(wider_plot_dict.values(), axis=1)
wider_plot_df

In [ ]:
for lib_mean in ['wildtype_mean', 'smg-5_mean', 'smg-6_mean']:
    wider_plot_df[f"{lib_mean}_diff"] = wider_plot_df[f'NMD_Targets_{lib_mean}'] - wider_plot_df[f'NMD_NonTargets_{lib_mean}']
    wider_plot_df[f"{lib_mean}_diff_frac"] = wider_plot_df[f'NMD_Targets_{lib_mean}'] / wider_plot_df[f'NMD_NonTargets_{lib_mean}']
wider_plot_df[[col for col in wider_plot_df.columns if 'diff' in col]]

# THE PROBLEM: I merged on lib types (N2, S5, S6) before I made the actual comparisons between NMD targets and non-NMD targets. I need to do this first, then merge. This will allow me to target the issue I noted above regarding variable depth, ligation, and xrn-1 knockdown!!

# THE PROBLEM:
I merged on lib types (N2, S5, S6) before I made the actual comparisons between NMD targets and non-NMD targets. I need to do this first, then merge. This will allow me to target the issue I noted above regarding variable depth, ligation, and xrn-1 knockdown!!

1. We want to compare the TSS ratio for NMD vs non_NMD targets in each library
2. We can take a mean of those ratios for each strain (ie. wt or smg-5)
3. Finally we can compare the mean ratios between strains, WT should look similar to smg-5 and smg-6 (if nothing is changing on the decapping front)

In [ ]:
# Before anything let's copy down code so we can regenerate the input df
summary_df = plotting_summary_df.copy()

lib_strain_dict = {'N2': 'wildtype',
                   'S5': 'smg-5',
                   'S6': 'smg-6',
                   'S7': 'smg-7'}

summary_df['strain'] = summary_df.lib.str[-2:].map(lib_strain_dict)

summary_df['total_near_tss'] = summary_df['adapted_near_tss'] + summary_df['unadapted_near_tss']
summary_df['total_near_stop'] = summary_df['adapted_near_stop'] + summary_df['unadapted_near_stop']
summary_df['total_reads'] = summary_df['total_adapted'] + summary_df['total_unadapted']

# There are a few different ways to do this!
summary_df['frac_near_tss'] = summary_df['total_near_tss'] / summary_df['total_reads']

summary_df['frac_near_tss'] = summary_df['adapted_near_tss'] / summary_df['total_reads']

summary_df

In [ ]:
# First we want to compare the TSS ratio for NMD vs non_NMD targets in each library
in_df = summary_df.copy()
set_dict = {'old': 1, 'newer': 2, 'third': 3, 'temp25c': 4}

in_df['rep'] = in_df.lib.str[:-2].map(set_dict)
in_df = in_df.set_index(['gene_name', 'nmd_target_type', 'strain', 'rep','lib'])[['frac_near_tss', 'total_reads']].sort_index()
inter_wide_df = in_df.loc[:, ['NMD_Targets', 'NMD_NonTargets'], :].unstack('nmd_target_type')
inter_wide_df

In [ ]:
# Second we can take a mean of those ratios for each strain (ie. wt or smg-5)
# First lets drop anything that doesn't have at least X reads for each:
min_reads = 50
semi_wide_df = inter_wide_df.copy()
semi_wide_df = semi_wide_df[semi_wide_df['total_reads'] >= min_reads].dropna(axis=0)
# So we want to divide the NMD_Targets by the NMD_NonTargets for each row of semi_wide_df
semi_wide_df['near_tss(nmd / non_nmd)'] = semi_wide_df['frac_near_tss']['NMD_Targets'] / semi_wide_df['frac_near_tss']['NMD_NonTargets']
semi_wide_df['total_total_reads'] = semi_wide_df['total_reads']['NMD_Targets'] + semi_wide_df['total_reads']['NMD_NonTargets']

In [ ]:
comp_box_df = semi_wide_df.copy().reset_index()
comp_box_df = comp_box_df[comp_box_df['near_tss(nmd / non_nmd)'] > 0]
comp_box_df['log2_near_tss(nmd / non_nmd)'] = np.log2(comp_box_df['near_tss(nmd / non_nmd)'])
comp_box_df.columns = ["_".join(a).strip('_') for a in comp_box_df.columns.to_flat_index()]
fig = px.box(comp_box_df,
             x='strain',
             y='near_tss(nmd / non_nmd)',
             log_y=True,
             # color='gene_name',
             points='all',
             hover_name='gene_name',
             hover_data=['rep',
                         'strain',
                         'lib',
                         'near_tss(nmd / non_nmd)',
                         'frac_near_tss_NMD_Targets',
                         'frac_near_tss_NMD_NonTargets',
                         ],
                )
fig.show()
comp_box_df

In [ ]:
# Finally we can compare the mean ratios between strains, WT should look similar to smg-5 and smg-6
final_df = semi_wide_df.copy()
final_df.columns = ["_".join(a).strip('_') for a in final_df.columns.to_flat_index()]
final_df = final_df[['frac_near_tss_NMD_Targets', 'frac_near_tss_NMD_NonTargets', 'NMD_Targets_frac', 'total_total_reads']]
final_df.sort_index()

In [ ]:
plot_df = final_df.reset_index()
window_size_for_search = 100
limits = [-0.1, 1.1] # [-0.0025, 0.03] # [-0.1, 1.1]
read_type = "Any Read" # "Adapted Reads" # Any Read

plot_df['name'] = plot_df['gene_name'] + ' ' + plot_df['strain'] + ' rep' + plot_df['rep'].astype(str)
fig = px.scatter(plot_df,
                 x='frac_near_tss_NMD_Targets',
                 y='frac_near_tss_NMD_NonTargets',
                 color='gene_name',
                 hover_name='name',
                 size='total_total_reads',
                 )
fig.update_layout(shapes = [{'type': 'line', 'line': {'dash': 'dot'}, 'yref': 'y', 'xref': 'x', 'y0': -1, 'y1': 2, 'x0': -1, 'x1': 2}],
                  title=f'Fraction of Total Reads Near TSS<br>(Counting {read_type} within {window_size_for_search}nt)',
                  )
fig.update_xaxes(constrain='domain', range=limits)  
fig.update_yaxes(scaleanchor= 'x', scaleratio= 1, range=limits)
fig.write_html(current_dir / 'plots' / 'decappingAndCleavage' / f"FractionOfTotalReadsNearTSS_{read_type.replace(' ', '')}.html")
fig.write_image(current_dir / 'plots' / 'decappingAndCleavage' / f"FractionOfTotalReadsNearTSS_{read_type.replace(' ', '')}.png")
fig.show()
plot_df

In [ ]:
final_df.unstack("strain").mean()

In [ ]:
final_df.unstack("strain").describe()

In [ ]:
fig = px.box(final_df.reset_index(),
             x='strain',
             y='NMD_Targets_frac',
             # color='gene_name',
             points='all',
             )
fig.show(renderer='firefox')
final_df.reset_index()

In [ ]:
semi_wide_df

In [ ]:
min_reads = 50

box_df = in_df.loc[:, :, :, :, ['NMD_Targets', 'NMD_NonTargets'], :].reset_index()
box_df['name'] = box_df['gene_name'] + ' ' + box_df['strain'] + ' rep' + box_df['rep'].astype(str)
box_df = box_df[box_df['total_reads'] >= min_reads].dropna(axis=0)
box_df = box_df[box_df['frac_near_tss'] > 0]

fig = px.box(box_df,
             x='strain',
             y='frac_near_tss',
             color='nmd_target_type',
             points='all',
             hover_name='name',
             log_y=True,
             )
fig.show(renderer='firefox')
box_df

In [ ]:
import scipy.stats as stats
from colorama import Fore, Style

def run_stats_intralib(df_in, compare_col, sig_cutoff=0.05, steps=[100, 10, 1],
                       strains=('wildtype', 'smg-5', 'smg-6'),
                       output_file: Path = None):
    test_results = {}
    output_string = ""
    for strain in strains:
        nmd_df = df_in.query("strain == @strain & nmd_target_type == 'NMD_Targets'")
        non_nmd_df = df_in.query("strain == @strain & nmd_target_type == 'NMD_NonTargets'")
        merge_df = nmd_df.merge(non_nmd_df,
                                on=('gene_name', 'strain', 'rep', 'lib', 'name'),
                                suffixes=('_nmd', '_non_nmd'))
        paired_test = stats.ttest_rel(merge_df[f'{compare_col}_nmd'], merge_df[f'{compare_col}_non_nmd'])
        unpaired_test = stats.ttest_ind(nmd_df[compare_col], non_nmd_df[compare_col])
        test_results[strain] = {'paired': paired_test, 'unpaired': unpaired_test}
    output_string += f"{'Strain':^8} | {'Paired p-val (df)':^15} | {'Unpaired p-val (df)':^15}\n"
    output_string += '-' * 50 + '\n'
    for strain, test_dict in test_results.items():
        paired_str = f" {test_dict['paired'].pvalue:^1.3e}   ({test_dict['paired'].df:>2})"
        unpaired_str = f"   {test_dict['unpaired'].pvalue:^1.3e}   ({int(test_dict['unpaired'].df):>2})"
        if test_dict['paired'].pvalue < sig_cutoff / steps[0]:
            paired_str = Fore.MAGENTA + paired_str + Style.RESET_ALL
        elif test_dict['paired'].pvalue < sig_cutoff / steps[1]:
            paired_str = Fore.RED + paired_str + Style.RESET_ALL
        elif test_dict['paired'].pvalue < sig_cutoff / steps[2]:
            paired_str = Fore.YELLOW + paired_str + Style.RESET_ALL
        if test_dict['unpaired'].pvalue < sig_cutoff / steps[0]:
            unpaired_str = Fore.MAGENTA + unpaired_str + Style.RESET_ALL
        elif test_dict['unpaired'].pvalue < sig_cutoff / steps[1]:
            unpaired_str = Fore.RED + unpaired_str + Style.RESET_ALL
        elif test_dict['unpaired'].pvalue < sig_cutoff / steps[2]:
            unpaired_str = Fore.YELLOW + unpaired_str + Style.RESET_ALL
        output_string += f"{strain:>8} | {paired_str} | {unpaired_str}\n"
    key_string = (f"Key: "
                  f"{Fore.MAGENTA}p < {sig_cutoff / steps[0]}{Style.RESET_ALL} "
                  f"{Fore.RED}p < {sig_cutoff / steps[1]}{Style.RESET_ALL} "
                  f"{Fore.YELLOW}p < {sig_cutoff / steps[2]}{Style.RESET_ALL}\n")
    print(output_string + key_string)
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    output_string = ansi_escape.sub('', output_string)
    if output_file is not None:
        with open(output_file, 'a') as f:
            f.write(output_string)
    return output_string

def run_stats_interlib(df_in, compare_col, sig_cutoff=0.05, steps=[100, 10, 1],
                       comparisons=(('wildtype', 'smg-5'), ('wildtype', 'smg-6'), ('smg-5', 'smg-6')),
                       nmd_type='NMD_NonTargets',
                       output_file=None):
    # Now let's do a similar comparison but between non-NMD-targets between libs
    test_results = {}
    output_string = ""
    for comp in comparisons:
        nmd_df = df_in.query("strain == @comp[0] & nmd_target_type == @nmd_type")
        non_nmd_df = df_in.query("strain == @comp[1] & nmd_target_type == @nmd_type")
        drop_cols = ['nmd_target_type', 'strain', 'lib', 'name']
        merge_df = nmd_df.drop(columns=drop_cols).merge(non_nmd_df.drop(columns=drop_cols), on=('gene_name', 'rep'), suffixes=('_lib1', '_lib2'))
        paired_test = stats.ttest_rel(merge_df[f'{compare_col}_lib1'], merge_df[f'{compare_col}_lib2'])
        unpaired_test = stats.ttest_ind(nmd_df[compare_col], non_nmd_df[compare_col])
        test_results[comp] = {'paired': paired_test, 'unpaired': unpaired_test}
    
    output_string += f"{'Comp':^14} | {'Paired p-val (df)':^15} | {'Unpaired p-val (df)':^15}\n"
    output_string += '-'*56 + '\n'
    
    for (lib1, lib2), test_dict in test_results.items():
        paired_str = f" {test_dict['paired'].pvalue:^1.3e}   ({test_dict['paired'].df:>2})"
        unpaired_str = f"   {test_dict['unpaired'].pvalue:^1.3e}   ({int(test_dict['unpaired'].df):>2})"
        if lib1 == 'wildtype':
            lib1 = 'wt'
        if test_dict['paired'].pvalue < sig_cutoff / steps[0]:
            paired_str = Fore.MAGENTA + paired_str + Style.RESET_ALL
        elif test_dict['paired'].pvalue < sig_cutoff / steps[1]:
            paired_str = Fore.RED + paired_str + Style.RESET_ALL
        elif test_dict['paired'].pvalue < sig_cutoff / steps[2]:
            paired_str = Fore.YELLOW + paired_str + Style.RESET_ALL
        if test_dict['unpaired'].pvalue < sig_cutoff / steps[0]:
            unpaired_str = Fore.MAGENTA + unpaired_str + Style.RESET_ALL
        elif test_dict['unpaired'].pvalue < sig_cutoff / steps[1]:
            unpaired_str = Fore.RED + unpaired_str + Style.RESET_ALL
        elif test_dict['unpaired'].pvalue < sig_cutoff / steps[2]:
            unpaired_str = Fore.YELLOW + unpaired_str + Style.RESET_ALL
        output_string += f"{lib1:>5} vs {lib2:<5} | {paired_str} | {unpaired_str}\n"
    key_string = (f"Key: "
                  f"{Fore.MAGENTA}p < {sig_cutoff / steps[0]}{Style.RESET_ALL} "
                  f"{Fore.RED}p < {sig_cutoff / steps[1]}{Style.RESET_ALL} "
                  f"{Fore.YELLOW}p < {sig_cutoff / steps[2]}{Style.RESET_ALL}\n")
    print(output_string + key_string)
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    output_string = ansi_escape.sub('', output_string)
    if output_file is not None:
        with open(output_file, 'a') as f:
            f.write(output_string)
    return output_string

run_stats_intralib(box_df, 'frac_near_tss')
print("\n\n")
run_stats_interlib(box_df, 'frac_near_tss')

In [ ]:
# Maybe the NMD targets have higher adapted_TSS fractions in the WT not because they have high numbers of adapted_TSS reads, but b/c they have lower numbers of the denominator (all reads)
# To remedy this, let's try comparing the adapted_TSS to the unadapted_TSS

# Additionally, it would be nice to do an adapted RPM comparison against Adapt TSS reads. But this needs more reads!!

# I need to circle WAY back to retain both adapt and unadapted TSSes.. UGH LETS DO IT
in_df = summary_df.copy()
set_dict = {'old': 1, 'newer': 2, 'third': 3, 'temp25c': 4}

in_df['rep'] = in_df.lib.str[:-2].map(set_dict)
in_df = in_df.set_index(['gene_name', 'strain', 'rep','lib', 'nmd_target_type']).sort_index()
better_df = in_df.loc[:, :, :, :, ['NMD_Targets', 'NMD_NonTargets'], :].drop(columns=['frac_near_tss'])
better_df

In [ ]:
min_reads = 1

# target_col = 'adapted_near_tss / unadapted_near_tss'
# target_col = 'adapted_near_tss / total_reads'

target_numerator = 'adapted_near_tss'

# target_denominator = 'total_reads'
# target_denominator = 'unadapted_near_tss'
# target_denominator = 'total_near_tss'
target_denominator = 'total_adapted'

target_col = f"{target_numerator} / {target_denominator}"


better_df['adapted_near_tss'] = better_df['adapted_near_tss'].fillna(0)
better_df['unadapted_near_tss'] = better_df['unadapted_near_tss'].fillna(0)
better_df[target_col] = better_df[target_numerator] / better_df[target_denominator]

box_df = better_df.copy().reset_index()
box_df = box_df[box_df['strain'] != 'smg-7']
box_df['name'] = box_df['gene_name'] + ' ' + box_df['strain'] + ' rep' + box_df['rep'].astype(str)
box_df = box_df[box_df['total_reads'] >= min_reads].dropna(axis=0)
box_df = box_df[box_df[target_col] > 0]

fig = px.box(box_df,
             x='strain',
             y=target_col,
             color='nmd_target_type',
             points='all',
             hover_name='name',
             log_y=True,
             )
save_friendly_target_col = target_col.replace(' ', '_').replace('/', '_vs_')
fig.write_html(current_dir / 'plots' / 'decapping' / f"boxPlot_{save_friendly_target_col}.html")
fig.write_image(current_dir / 'plots' / 'decapping' / f"boxPlot_{save_friendly_target_col}.png")
fig.show(renderer='firefox')

output_file = current_dir / 'plots' / 'decapping' / f'boxPlotStats_{save_friendly_target_col}.txt'
output_file.touch(exist_ok=True)
with open(output_file, 'w') as f:
    f.write(f"Stats for {target_col}\n")
    f.write("\nIntra Library Stats w/ NMD Targets:\n")
print("Targets")
run_stats_interlib(box_df, target_col, nmd_type='NMD_Targets', output_file=output_file)
with open(output_file, 'a') as f:
    f.write("\nIntra Library Stats w/ Non Targets:\n")
print("NonTargets")
run_stats_interlib(box_df, target_col, nmd_type='NMD_NonTargets', output_file=output_file)

with open(output_file, 'a') as f:
    f.write("\nInter Library Stats:\n")
run_stats_intralib(box_df, target_col, output_file=output_file)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

def run_box_and_ttests(input_df: pd.DataFrame,
                       target_num: str, target_denom: str,
                       min_reads: int = 1,
                       output_dir: Path = current_dir / 'plots' / 'decapping'):
    target_col = f"{target_num} / {target_denom}"
    input_df
    
    input_df['adapted_near_tss'] = input_df['adapted_near_tss'].fillna(0)
    input_df['unadapted_near_tss'] = input_df['unadapted_near_tss'].fillna(0)
    input_df[target_col] = input_df[target_num] / input_df[target_denom]
    
    box_df = input_df.copy().reset_index()
    box_df = box_df[box_df['strain'] != 'smg-7']
    box_df['name'] = box_df['gene_name'] + ' ' + box_df['strain'] + ' rep' + box_df['rep'].astype(str)
    box_df = box_df[box_df['total_reads'] >= min_reads].dropna(axis=0)
    box_df = box_df[box_df[target_col] > 0]
    
    fig = px.box(box_df,
                 x='strain',
                 y=target_col,
                 color='nmd_target_type',
                 points='all',
                 hover_name='name',
                 log_y=True,
                 )
    save_friendly_target_col = target_col.replace(' ', '_').replace('/', '_vs_')
    fig.write_html(output_dir / f"boxPlot_{save_friendly_target_col}.html")
    fig.write_image(output_dir / f"boxPlot_{save_friendly_target_col}.png")
    fig.show(renderer='firefox')
    
    output_file = output_dir / f'boxPlotStats_{save_friendly_target_col}.txt'
    output_file.touch(exist_ok=True)
    with open(output_file, 'w') as f:
        f.write(f"Stats for {target_col}\n")
        f.write("\nIntra Library Stats w/ NMD Targets:\n")
    print("Targets")
    run_stats_interlib(box_df, target_col, nmd_type='NMD_Targets', output_file=output_file)
    with open(output_file, 'a') as f:
        f.write("\nIntra Library Stats w/ Non Targets:\n")
    print("NonTargets")
    run_stats_interlib(box_df, target_col, nmd_type='NMD_NonTargets', output_file=output_file)
    
    with open(output_file, 'a') as f:
        f.write("\nInter Library Stats:\n")
    run_stats_intralib(box_df, target_col, output_file=output_file)

for target_denominator in ['total_reads', 'unadapted_near_tss', 'total_adapted']:
    print(f"\n\nUsing {target_denominator} for denominator...\n")
    run_box_and_ttests(better_df, 'adapted_near_tss', target_denominator)
print("Done!")

In [ ]:
# Let's see if we can make quick bar plots for individual gene species